# Introduction to AI - Lab 11

## Genetic Path Planning

### Motivation
In this lab, we will explore the application of Genetic Algorithms (GAs) to the problem of path planning, which is a crucial aspect of navigation systems in robotics. We will implement a genetic path planner and evaluate its performance.

### Components of Genetic Path Planning
- **Chromosomes:** Represent possible solutions (paths) in the search space.
- **Fitness Function:** Evaluates how good a solution is.
- **Selection:** Chooses the best solutions for reproduction.
- **Crossover:** Combines parts of two solutions to create offspring.
- **Mutation:** Introduces random changes to solutions to maintain diversity.

## Task: Implementing Genetic Path Planning

### Step 1: Define the Problem
We will use a grid-based environment with obstacles. The objective is to find a path from a start point to an end point while avoiding obstacles.

In [1]:
!pip install descartes

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon
from descartes import PolygonPatch

class MyPoint:
    def __init__(self, *args):
        self.point = Point(*args)
        self.x, self.y = self.point.x, self.point.y

    def __add__(self, other):
        return MyPoint(self.x + other.x, self.y + other.y)

    def scale(self, ratio):
        return MyPoint(self.x * ratio, self.y * ratio)

    def get_xy(self):
        return self.x, self.y

    def rotate(self, theta):
        c, s = np.cos(theta), np.sin(theta)
        r = np.array([[c, -s], [s, c]])
        new_xy = list(np.matmul(r, self.get_xy()))
        return MyPoint(new_xy[0], new_xy[1])

### Step 2: Define the Robot and Environment
The robot will navigate in a grid with obstacles. We will define the robot, obstacles, and cost functions to evaluate the path.

In [3]:
class Robot:
    def __init__(self, start_point, end_point, grid_num, obstacles):
        self.start = start_point
        self.stop = end_point
        self.grid_num = grid_num
        self.obstacles = obstacles
        self.create_path()

    def create_path(self):
        self.path = [self.start] + [MyPoint(0, 0) for _ in range(self.grid_num)] + [self.stop]

    def update_path(self, points):
        self.path = [self.start] + points + [self.stop]

    def get_cost(self, genes):
        points = [MyPoint(g[0], g[1]) for g in genes]
        self.update_path(points)
        cost = 0
        for i in range(len(self.path) - 1):
            line = LineString([self.path[i].get_xy(), self.path[i + 1].get_xy()])
            for obs in self.obstacles:
                if line.intersects(Polygon([p.get_xy() for p in obs.points])):
                    cost += 100
            cost += line.length
        # Add penalty for complexity (number of segments)
        cost += len(self.path) * 10
        return cost

class Obstacle:
    def __init__(self, center_point, size=1.0):
        corners = [MyPoint(-1, -1), MyPoint(-1, 1), MyPoint(1, 1), MyPoint(1, -1)]
        scaled_corners = [p.scale(size) + center_point for p in corners]
        self.points = scaled_corners

    def get_drawable(self, color):
        return plt.Polygon([(p.x, p.y) for p in self.points], color=color)

def plot_environment(robot, title="Path Planning"):
    fig, ax = plt.subplots()
    for obs in robot.obstacles:
        try:
            patch = PolygonPatch(Polygon([p.get_xy() for p in obs.points]), fc='red', alpha=0.5)
            ax.add_patch(patch)
        except Exception as e:
            print(f"Error plotting obstacle: {e}")
    path_x, path_y = zip(*[p.get_xy() for p in robot.path])
    ax.plot(path_x, path_y, 'bo-', label='Path')
    ax.plot(robot.start.x, robot.start.y, 'go', label='Start')
    ax.plot(robot.stop.x, robot.stop.y, 'ro', label='Stop')
    plt.title(title)
    plt.legend()
    plt.show()

# Debugging to ensure obstacles are correctly formed
for obs in obstacles:
    print("Obstacle points:", [p.get_xy() for p in obs.points])

### Step 3: Define the Genetic Algorithm Components
We will define the chromosome, mutation, crossover, and fitness functions.

In [4]:
class Chromosome:
    def __init__(self, genes_len=10, gene_pool_min=-5, gene_pool_max=5, genes=None):
        if genes is None:
            self.genes = np.random.uniform(gene_pool_min, gene_pool_max, (genes_len, 2))
        else:
            self.genes = genes

    def mutate(self, gene_pool_min, gene_pool_max):
        mutation_index = np.random.randint(0, len(self.genes))
        new_genes = self.genes.copy()
        new_genes[mutation_index] = np.random.uniform(gene_pool_min, gene_pool_max, 2)
        return Chromosome(genes_len=len(new_genes), genes=new_genes)

    def crossover(self, other):
        cross_point = np.random.randint(1, len(self.genes) - 1)
        offspring1 = np.concatenate((self.genes[:cross_point], other.genes[cross_point:]))
        offspring2 = np.concatenate((other.genes[:cross_point], self.genes[cross_point:]))
        return Chromosome(genes_len=len(offspring1), genes=offspring1), Chromosome(genes_len=len(offspring2), genes=offspring2)

    def get_genes(self):
        return list(self.genes).copy()

In [5]:
class GA:
    def __init__(self, chr_size, talent_size):
        self.chr_size = chr_size
        self.talent_size = talent_size
        self.population = []
        self.top = {"cost_value": float('Inf'), "chr": []}

    def reset_top(self):
        self.top = {"cost_value": float('Inf'), "chr": []}

    def reset(self, pop_size):
        self.population = []
        self.gen_population(gene_pool_min=-3, gene_pool_max=3, pop_size=pop_size)

    def append_population(self, population):
        self.population = self.population + population

    def change_population(self, pop):
        del (self.population[int(len(self.population) / 2):])
        self.append_population(pop)

    def gen_population(self, gene_pool_max, gene_pool_min, pop_size):
        for p in range(pop_size):
            self.population.append(Chromosome(self.chr_size, gene_pool_min, gene_pool_max))
        return self.population

    def mutation(self, num, gene_pool_min, gene_pool_max):
        if num > len(self.population):
            raise ValueError("number of mutation is higher than population")
        mutated = []
        mutate_indexes = np.random.randint(0, len(self.population), num)
        for mutate_index in mutate_indexes:
            mutated = mutated + [self.population[mutate_index].mutate(gene_pool_min, gene_pool_max)]
        return mutated

    def crossover(self, num):
        crossover_pop = []
        for i in range(num):
            s = list(np.random.randint(0, len(self.population), 2))
            crossover_tuple = self.population[s[0]].crossover(self.population[s[1]])
            crossover_pop = crossover_pop + list(crossover_tuple)
        return crossover_pop

    def calc_fitness(self, func, pop=None):
        if pop is None:
            pop = []
        if len(pop) == 0:
            fitness_list = [func(chr_.get_genes()) for chr_ in self.population]
        else:
            fitness_list = [func(chr_.get_genes()) for chr_ in pop]
        sorted_list = sorted(zip(fitness_list, self.population), key=lambda f: f[0])
        sorted_chromosome = [s[1] for s in sorted_list]
        top_fitness = sorted_list[0][0]
        if self.top["cost_value"] > top_fitness:
            self.top["cost_value"] = top_fitness
            self.top["chr_"] = sorted_list[0][1]
        return sorted_chromosome, top_fitness

### Step 4: Run the Genetic Algorithm
We will set up the main loop of the GA and iterate through its elements.

In [6]:
run_index = 1
flag = True
grid_size = 15
pop_size = 20
start = MyPoint(0, 0)
end = MyPoint(10, 10)
obstacles = [Obstacle(MyPoint(5, 5), size=1.5)]
robot = Robot(start, end, grid_size, obstacles)
ga = GA(chr_size=grid_size, talent_size=3)
g = ga.gen_population(gene_pool_min=-5, gene_pool_max=5, pop_size=pop_size)

def ga_iterate(num, mutate_chance=0.8, mutate_min=-15, mutate_max=15):
    global flag
    cost = []
    for i in range(num):
        best_path, most_fit = ga.calc_fitness(robot.get_cost)
        cost.append(most_fit)
        ga.population = best_path
        crossovered = ga.crossover(int(pop_size / 2))
        if flag:
            ga.append_population(crossovered)
            flag = False
        else:
            ga.change_population(crossovered)

        a = np.random.uniform(0, 1, 1)
        if a < mutate_chance:
            mutated = ga.mutation(pop_size, mutate_min, mutate_max)
            ga.change_population(mutated)
        # Visualize the path at each generation
        robot.update_path([MyPoint(*p) for p in best_path[0].get_genes()])
        plot_environment(robot, title=f"Generation {i+1}")
    return best_path, cost

# Run GA and visualize results
best_path, cost = ga_iterate(50)
robot.update_path([MyPoint(*p) for p in best_path[0].get_genes()])
plot_environment(robot, title="Final Path after Genetic Algorithm")

# Plot cost over generations
plt.plot(cost)
plt.title("Cost over Generations")
plt.xlabel("Generation")
plt.ylabel("Cost")
plt.show()

### Conclusion
In this lab, we implemented a Genetic Algorithm to solve the path planning problem. We explored key operations such as selection, crossover, and mutation, and observed how the population evolves over generations to find the optimal path. The visualization shows the final path and the cost reduction over generations.